
# Add an Output System

This example demonstrates how to build and register a new output system in
MyoGestic.  An output system receives model predictions and routes them to
an external destination -- a visual interface, prosthetic hardware, game
engine, or anything else that consumes EMG-decoded outputs.

Output systems are created **per active visual interface** when a trained
model is loaded in the Online tab.  Each output system is registered with
a name that matches the short name of its target VI (e.g., ``"VHI"``,
``"KHI"``, ``"VCI"``).

## Multi-VI Architecture
MyoGestic supports multiple active visual interfaces simultaneously.
When a model is loaded, the Online protocol creates output systems only
for the VIs that are currently active:

```python
# Inside OnlineProtocol._load_model():
active_vis = self._main_window.active_visual_interfaces  # dict[str, VisualInterface]
for vi_name, vi in active_vis.items():
    output_system = CONFIG_REGISTRY.output_systems_map[vi_name](
        self._main_window, model.is_classifier
    )
```
To access a specific VI from within your output system:

```python
vi = self._main_window.active_visual_interfaces.get("VHI")
```
## Required Methods
Your output system must inherit from `output_system_template` and
implement:

- :meth:`~myogestic.gui.widgets.templates.OutputSystemTemplate._process_prediction__classification`
- :meth:`~myogestic.gui.widgets.templates.OutputSystemTemplate._process_prediction__regression`
- :meth:`~myogestic.gui.widgets.templates.OutputSystemTemplate.send_prediction`
- :meth:`~myogestic.gui.widgets.templates.OutputSystemTemplate.close_event`

The base class automatically selects the right processing method based
on ``prediction_is_classification`` and exposes it as
:attr:`~myogestic.gui.widgets.templates.OutputSystemTemplate.process_prediction`.

.. admonition:: Add your output system in :mod:`~myogestic.user_config`

   Keep custom output system registrations in :mod:`~myogestic.user_config` to stay
   separate from core MyoGestic code.


### Step 1: Define Your Custom Output System



In [ ]:
from typing import Any
from myogestic.gui.widgets.templates.output_system import OutputSystemTemplate


class MyCustomOutputSystem(OutputSystemTemplate):
    """A simple example output system for demonstration.

    This class shows how to inherit from :class:`~myogestic.gui.widgets.templates.OutputSystemTemplate` and
    implement the required abstract methods.  It sends predictions to
    the application logger for demonstration purposes.
    """

    def __init__(self, main_window, prediction_is_classification: bool) -> None:
        super().__init__(main_window, prediction_is_classification)
        # Initialize sockets, hardware connections, timers, etc. here.
        #
        # To access the outgoing signal of a specific VI:
        #   vi = self._main_window.active_visual_interfaces.get("VHI")
        #   self._outgoing_signal = vi.outgoing_message_signal

    def _process_prediction__classification(self, prediction: Any) -> bytes:
        """Convert a classification label to bytes for transmission."""
        return f"Class: {prediction}".encode("utf-8")

    def _process_prediction__regression(self, prediction: Any) -> bytes:
        """Convert regression values to bytes for transmission."""
        return str([float(x) for x in prediction]).encode("utf-8")

    def send_prediction(self, prediction: Any) -> None:
        """Send the processed prediction to the output destination."""
        processed = self.process_prediction(prediction)
        # Replace with your actual send logic (UDP, serial, etc.)
        self._main_window.logger.print(f"Sending: {processed}")

    def close_event(self, event) -> None:
        """Clean up resources (sockets, timers, etc.)."""
        pass

### Step 2: Register the Output System in CONFIG_REGISTRY
The ``name`` must match the short name of the target VI.  For a
standalone output system (not tied to a specific VI), use any unique
name -- it will be instantiated when a model is loaded.



In [ ]:
from myogestic.utils.config import CONFIG_REGISTRY

CONFIG_REGISTRY.register_output_system(
    name="MyCustomOutputSystem", output_system=MyCustomOutputSystem
)

### Reference: Virtual Hand Interface Output System
The VHI output system demonstrates a real-world implementation that:

- Validates that the VHI is among the active visual interfaces.
- Maps classification labels (0-8) to predefined hand poses.
- Converts regression predictions to a float list string.
- Sends the processed prediction via the VI's outgoing UDP signal.

.. literalinclude:: /../../myogestic/gui/widgets/visual_interfaces/virtual_hand_interface/output_interface.py
   :language: python
   :lineno-match:
   :caption: VHI Output System -- full implementation

